In [1]:
import sys
import random
import numpy as np
from scipy.misc import imread
import tensorflow as tf
 
num = 2000
    
img_list = ['zero_'+str(i)+'.png' for i in range(1,num+1)]
lbl_list = [0 for i in range(num)]
img_list = img_list+['one_'+str(i)+'.png' for i in range(1,num+1)]
lbl_list = lbl_list+[1 for i in range(num)]

# print img_list,lbl_list

In [2]:
import os
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "AIresearch.settings")
import django
django.setup()
from basic_model.models import Mnistimage


def trainedFiles():
    trainedObjects = Mnistimage.objects.exclude(trained_label=3)
    images = trainedObjects.values('image')
    labels = trainedObjects.values('trained_label')
    images = [i['image'] for i in list(images)]
    labels = [i['trained_label']-1 for i in list(labels)]

    return images,labels
img_list,lbl_list = list(trainedFiles())

In [3]:
print img_list
print lbl_list

[u'MNIST_images/one_6.png', u'MNIST_images/one_17.png', u'MNIST_images/one_28.png', u'MNIST_images/one_23.png', u'MNIST_images/one_10.png', u'MNIST_images/one_18.png', u'MNIST_images/one_30.png', u'MNIST_images/zero_5156.png', u'MNIST_images/one_13.png', u'MNIST_images/zero_3607.png', u'MNIST_images/zero_1.png', u'MNIST_images/zero_13.png']
[1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0]


In [4]:
def data_provider(img_list,lbl_list,n):
    
    images = np.zeros((n,28,28,1))
    labels = np.zeros((n,2))
    
    n_files = len(img_list)
    n = min(n,n_files)
    
    lst = np.arange(n_files)
    random.shuffle(lst)
    lst = lst[:n]
    
    for i in range(n):
        images[i,:,:,0] = imread('./media/'+img_list[lst[i]])
        labels[i,lbl_list[lst[i]]] =1 
        
    return images,labels

In [5]:
x_in = tf.placeholder(tf.float32, shape=[None,28,28,1])
y_ = tf.placeholder(tf.float32, shape=[None,2])
keep_prob = tf.placeholder(tf.float32)

x = tf.layers.conv2d(x_in,filters=5,kernel_size=3,strides=(1, 1),padding='same',
            activation=tf.nn.relu)
x = tf.layers.max_pooling2d(x,pool_size=2,strides=2)

x = tf.layers.conv2d(x,filters=5,kernel_size=3,strides=(1, 1),padding='same',
        activation=tf.nn.relu)
x = tf.layers.max_pooling2d(x,pool_size=2,strides=2)

x = tf.contrib.layers.flatten(x)
x = tf.nn.dropout(x, keep_prob)

y_conv = tf.layers.dense(x,2,activation=tf.nn.softmax)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [7]:
def train(img_list,lbl_list,restore=1,model_add='model'):
    sess = tf.InteractiveSession()
    saver = tf.train.Saver()

    if restore:
#         tf.reset_default_graph()
        saver.restore(sess, model_add+'/model')
    else:
        init = tf.global_variables_initializer()
        sess.run(init)

    for i in range(100):
        xp,yp = data_provider(img_list,lbl_list,100)
        if i % 10 == 0:
            train_accuracy = accuracy.eval(feed_dict={x_in: xp, y_: yp, keep_prob: 1.0})
            print('{:d}, {:2.2f}'.format(i, train_accuracy))
        train_step.run(feed_dict={x_in: xp, y_: yp, keep_prob: 0.8})

    saver.save(sess, model_add+'/model')
    
train(img_list,lbl_list,restore=1)

INFO:tensorflow:Restoring parameters from model/model
0, 0.05
10, 0.06
20, 0.06
30, 0.07
40, 0.07
50, 0.07
60, 0.07
70, 0.07
80, 0.07
90, 0.07


In [12]:
def predict(img_path,model_add='model'):
    images = np.zeros((1,28,28,1))
    images[0,:,:,0] = imread(img_path)

    sess = tf.InteractiveSession()
    saver = tf.train.Saver()
    saver.restore(sess, model_add+'/model')

    y_out = sess.run(y_conv,feed_dict={x_in: images, keep_prob: 1.0})
    return np.argmax(y_out)

In [13]:
img_path = './media/MNIST_images/zero_15.png'
predict(img_path,model_add='model')

INFO:tensorflow:Restoring parameters from model/model


0

In [34]:
def predict_all(model_add='model'):

    all_objects = Mnistimage.objects.all()
    images_lst = all_objects.values('image')
    images_id = all_objects.values('imageid')
    labels = all_objects.values('trained_label')
    images_lst = [i['image'] for i in list(images_lst)]
    images_id = [i['imageid'] for i in list(images_id)]
    labels = [i['trained_label']-1 for i in list(labels)]
    
    n_img = len(labels)
    
    images = np.zeros((n_img,28,28,1))
    
    
    for i in range(n_img):
        images[i,:,:,0] = imread('./media/'+images_lst[i]) 
    

    sess = tf.InteractiveSession()
    saver = tf.train.Saver()
    saver.restore(sess, model_add+'/model')

    y_out = sess.run(y_conv,feed_dict={x_in: images, keep_prob: 1.0})
    y_pred = np.argmax(y_out,axis=1)
    for i in images_id:
        mnistimage=Mnistimage.objects.get(imageid=i)
        mnistimage.predicted_label=1
        mnistimage.save()
    
    
    return np.sum(1.*(y_pred==labels))/n_img

predict_all()

INFO:tensorflow:Restoring parameters from model/model


0.58333333333333337

In [39]:
import random
def trainedFiles(n):
    trainedObjects = Mnistimage.objects.filter(trained_label=3)
    images = trainedObjects.values('image')
    images_id = trainedObjects.values('imageid')
    labels = trainedObjects.values('trained_label')
    
    images = [i['image'] for i in list(images)]
    images_id = [i['imageid'] for i in list(images_id)]
    
    n_img = len(labels)
    r_lst = np.arange(n_img)
    random.shuffle(r_lst)
    r_inds = r_lst[:n]
    
    for i in r_inds:
    
    
        print images[i]
    
        if 'one' in images[i]:
            true_lbl = 1
        else:
            true_lbl = 0
    
        mnistimage=Mnistimage.objects.get(imageid=images_id[i]) 
        mnistimage.predicted_label=true_lbl
        mnistimage.save()
    

trainedFiles(1)